In [1]:
def createlinks(start,end):
    base = 'https://en.wikipedia.org/wiki/'
    tails=[]
    for i in range(int(start),int(end)):
        tails.append('{}_Mosconi_Cup'.format(i))
    links=[base+tail for tail in tails]
    return(links)

In [454]:
def createtables(url):
    page = requests.get(url)
    tree = html.fromstring(page.text)
    tables = tree.xpath('//table')
    tables=[table for table in tables if(table[0].text_content().replace('\n','')=='Results')]
    return(tables)

In [ ]:
url = links[10]
tables = createtables(url)
for table in tables:
    nrows = table.

In [487]:
def createdf(tables):
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']

    for i in range(len(tables)+1):
        for j in range(1,6):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s", element)
                if match:
                    l.append(list(match.groups()))
            except:
                pass
    df=pd.DataFrame(l,columns=col)
    return(df)

In [664]:
def main():
    from lxml import html
    import requests
    import pandas as pd
    df = pd.DataFrame()
    links = createlinks('1998','2014')
    for url in links:
        tables = createtables(url)
        newdf = createdf(tables) 
        df=df.append(newdf)
    return(df)

In [710]:
pkl=r'C:\Users\andrew\Desktop\mosconi\mosconi_1998_2015_pkls'
df.to_pickle(pkl)

Missing 2014 data. Can get it from: http://www.matchroompool.com/mosconi-cup/mosconi-cup-xxi/#fixtures
compare stats here:
http://forums.azbilliards.com/showthread.php?t=439775&page=4

In [569]:
import re
l=[]
with open('manual_scrape_cleaned_2014.txt','r') as rf:
    for line in rf:
        match = re.match(r"(\w+)\s(.+)\s([0-9])\s?[-]\s?([0-9])\s?(.+)", line)
        if match:
            l.append(list(match.groups()))

In [583]:
df=pd.read_pickle(pkl)

In [670]:
df2=pd.DataFrame(l,columns=col)

In [671]:
frames=[df,df2]
df=pd.concat(frames)

In [672]:
df['Europe_won']=df.European_score>df.American_score

In [674]:
pkl=r'C:\Users\andrew\Desktop\mosconi\mosconi_1998_2014_inc_pkls'
df.to_pickle(pkl)

In [2]:
cols= ['Format','European_player','European_score','American_score','American_player','Europe_won']
df=df[cols]

NameError: name 'df' is not defined

2015 Data is backwards, need to fix the order (european player should be first)

In [705]:
def createdf_bw(tables):
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']

    for i in range(len(tables)+1):
        for j in range(1,6):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s", element)
                if match:
                    print('ok')
                    lst=[match.group(1), match.group(5), match.group(4), match.group(3), match.group(2)]
                    print(lst)
                    l.append(lst)
                else:
                    print('fail')
            except:
                pass
    print(l)
    df=pd.DataFrame(l,columns=col)
    return(df)

In [701]:
def main_bw():
    from lxml import html
    import requests
    import pandas as pd
    df = pd.DataFrame()
    links = createlinks('2015','2016')
    for url in links:
        tables = createtables(url)
        newdf = createdf_bw(tables) 
        df=df.append(newdf)
    return(df)

In [706]:
df2015= main_bw()

ok
['Teams', 'Team Europe', '4', '5', 'Team USA']
ok
['Doubles', 'Karl Boyes Nick van den Berg', '5', '4', 'Corey Deuel Mike Dechaine']
ok
['Singles', 'Niels Feijen', '5', '2', 'Justin Bergman']
ok
['Doubles', 'Darren Appleton Albin Ouschan', '1', '5', 'Shane Van Boening Skyler Woodward']
ok
['Singles', 'Darren Appleton', '5', '1', 'Shane Van Boening']
ok
['Doubles', 'Niels Feijen Nick van den Berg', '3', '5', 'Justin Bergman Skyler Woodward']
ok
['Singles', 'Albin Ouschan', '5', '3', 'Mike Dechaine']
ok
['Doubles', 'Darren Appleton Karl Boyes', '5', '2', 'Shane Van Boening Corey Deuel']
ok
['Singles', 'Karl Boyes', '5', '3', 'Skyler Woodward']
ok
['Doubles', 'Darren Appleton Nick van den Berg', '4', '5', 'Shane Van Boening Mike Dechaine']
ok
['Doubles', 'Niels Feijen Albin Ouschan', '5', '1', 'Justin Bergman Corey Deuel']
ok
['Singles', 'Nick van den Berg', '4', '5', 'Corey Deuel']
ok
['Doubles', 'Karl Boyes Albin Ouschan', '5', '2', 'Mike Dechaine Skyler Woodward']
ok
['Singles', 'Ni

In [708]:
frames=[df,df2015]
df=pd.concat(frames)

In [665]:
df=main()

Next steps: Rescrape the data. 

1. Need to include the year as a multiindex and concatenate the frames per year.
1. Need to reconfigure the number of rows to possibly extend beyond 5. Have to dynamically learn the number of rows in the    given table and then use that number instead of hard coding it.
3. Need to collect the 'team' data each year. I.e. what players are on each team so that I can do analysis on the 'team' matches and include that in the final presentations.
4. Scrape location or just see if it changes every year

1. Reconfigure number of rows dynamically.

In [5]:
def createlinks(start,end):
    base = 'https://en.wikipedia.org/wiki/'
    tails=[]
    for i in range(int(start),int(end)):
        tails.append('{}_Mosconi_Cup'.format(i))
    links=[base+tail for tail in tails]
    return(links)

In [487]:
def createdf(tables):
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']

#     for i in range(len(tables)+1):
#         for j in range(1,6):
#             try:
#                 element=tables[i][j].text_content().replace("\n", " ")
#                 match = re.match(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s", element)
#                 if match:
#                     l.append(list(match.groups()))
#             except:
#                 pass
#     df=pd.DataFrame(l,columns=col)
#     return(df)

In [6]:
def makesoup(url):
	import requests, bs4, re
	res = requests.get(url)
	res.raise_for_status()
	soup = bs4.BeautifulSoup(res.text, "lxml")
	return(soup)

In [10]:
links = createlinks(1998,2014)

In [13]:
links[10]

'https://en.wikipedia.org/wiki/2008_Mosconi_Cup'

In [14]:
from bs4 import BeautifulSoup as bs

url = links[10]
soup = makesoup(url)
tables = soup.findAll('table')
l = len(tables[3].findAll("tr"))-3

l


4